In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")

loc = geolocator.geocode('Champagne, France')


In [ ]:
# Import Meteostat library
from meteostat import Stations, Daily

# Get nearby weather stations
stations = Stations()
stations = stations.nearby(loc.latitude, loc.longitude)
station = stations.fetch(1)

In [ ]:
from datetime import datetime, timedelta
import numpy as np

# Set time period
start = datetime(2000, 1, 1)
end = datetime(2018, 12, 31)

# Get Daily data
data = Daily(station.index.values[0], start, end)
data = data.fetch()
data = data.reset_index()
data = data[['time','tavg', 'tmin', 'tmax']]


In [ ]:
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month

In [ ]:
data_month_year = data.groupby(['year', 'month'])[['tavg', 'tmin', 'tmax']].mean().reset_index()

In [ ]:
data_month = data_month_year.groupby('month')[['tavg', 'tmin', 'tmax']].mean().reset_index()
data_month.columns = ['month', 'tavg_month', 'tmin_month', 'tmax_month']
data_month_year = data_month_year.merge(data_month, how = 'left', on = 'month')

In [ ]:
data_month_year['tflux_month'] = data_month_year['tavg'] - data_month_year['tavg_month']

In [ ]:
convert_months = {
    1:"Jan",
    2:"Feb",
    3:"Mar",
    4:"Apr",
    5:"May",
    6:"Jun", 
    7:"July",
    8:"Aug",
    9:"Sep",
    10:"Oct",
    11:"Nov",
    12:"Dec"
}
data_month_year['month'] = data_month_year['month'].replace(convert_months)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plot_data = data_month_year[data_month_year['year'] == 2006]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x = plot_data['month'], y = plot_data['tflux_month'], name = "2005"))
fig.update_layout(
    shapes = [
    {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': 'Mar',
            'y0': 0,
            'x1': 'Oct',
            'y1': 1,
            'fillcolor': '#ffffe0',
            'opacity': 0.5,
            'line': {
                'width': 0,
            }
    
    }
    ]
)
#fig.add_trace(go.Scatter(x = plot_data['month'], y = plot_data['tmax']), secondary_y=True)
#fig.add_trace(go.Scatter(x = plot_data['month'], y = plot_data['tmin']), secondary_y=True)